# Extracted WLCs

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob as glob
import matplotlib as mpl
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import numpy as np
import pandas as pd
import seaborn as sns

import bz2
import corner
import json
import pathlib
import pickle
import utils
import warnings

from astropy import constants as const
from astropy import units as uni
from astropy.io import ascii, fits
from astropy.time import Time
from mpl_toolkits.axes_grid1 import ImageGrid

# Default figure dimensions
FIG_WIDE = (11, 5)
FIG_LARGE = (8, 11)

# Figure style
sns.set(style="ticks", palette="colorblind", color_codes=True, context="talk")
params = utils.plot_params()
plt.rcParams.update(params)

## [Dowload data](https://www.dropbox.com/sh/f6rqfis0rdd0w1r/AAADJF0JI-YEx1cS9oCiwKP6a?dl=1) (same dataset used for extracted spectra)

Unzip this into a folder named `data` in the same level as this notebook

## Finish set-up

In [3]:
! ln -srf data/extracted_spectra data/extracted_wlcs

## Load

In [153]:
# Load
data_dir = "data/extracted_wlcs"
LCs_pickle = "LCs_hp23_bins.pbz2"
data_dict = {
    "Transit 1": {
        "data": utils.decompress_pickle(
            f"{data_dir}/ut160621_a15_20_noflat/{LCs_pickle}"
        ),
        "bad_idxs_user": "[239, 240, 241, 256, 276, 278, 283, 284, 288:299]",
        "t0": 2457561.84167,
    },
    "Transit 2": {
        "data": utils.decompress_pickle(
            f"{data_dir}/ut170609_a15_25_noflat/{LCs_pickle}"
        ),
        "bad_idxs_user": "[10]",
        "t0": 2457914.79167,
    },
    "Transit 3": {
        "data": utils.decompress_pickle(
            f"{data_dir}/ut180603_a15_25_noflat/{LCs_pickle}"
        ),
        "bad_idxs_user": "[0:17]",
        "t0": 2458273.80556,
    },
    "Transit 4": {
        "data": utils.decompress_pickle(
            f"{data_dir}/ut180620_a15_25_noflat/{LCs_pickle}"
        ),
        "bad_idxs_user": "[93, 94, 149, 157:171]",
        "t0": 2458290.78472,
    },
    "Transit 5": {
        "data": utils.decompress_pickle(
            f"{data_dir}/ut180821_a15_25_noflat/{LCs_pickle}"
        ),
        "bad_idxs_user": "[4:6, 51, 53, 54, 56:59, 64, 66, 68, 69, 72, 77:80, 87:94, 96:98, 101, 108:119, 120, 129:131, 133, 174:181]",
        "t0": 2458352.64028,
    },
}

## Plot

In [179]:
comps = ["comp4", "comp5"]
ncomps = len(comps)

fig, axes = plt.subplots(
    len(data_dict), ncomps, sharex=True, sharey=True, figsize=FIG_LARGE
)

if axes.ndim == 1:
    axes = axes.reshape(1, axes.size)
colors = sns.color_palette()
for i, (transit_name, transit_info) in enumerate(data_dict.items()):
    data = transit_info["data"]
    t0 = transit_info["t0"]

    bad_idxs_both_comps = []
    for ax, comp in zip(axes[i, :], comps):
        p, bad_idxs = utils.plot_divided_wlcs(
            ax,
            data,
            t0=t0,
            ferr=0.001,
            comps_to_use=[comp],
            div_kwargs={"fmt": ".", "lw": 0.5, "mew": 0.0, "ms": 3, "c": colors[i]},
            bad_div_kwargs={"fmt": ".", "lw": 0.5, "mec": "k", "ms": 6, "c": "w"},
        )
        bad_idxs_both_comps.append(bad_idxs)

        ax.annotate(
            rf"{transit_name}/{comp}",
            xy=(0.05, 0.85),
            xycoords="axes fraction",
            fontsize=12,
            color=colors[i],
            weight="bold",
        )

        ax.set_ylim(0.96, 1.04)
        ax.yaxis.set_ticks([0.96, 0.98, 1.00, 1.02, 1.04])
    
    # Highlight outliers
    #print(transit_name)
    #super_bad = np.union1d(bad_idxs_both_comps[0], bad_idxs_both_comps[1])
    #print(repr(super_bad))
    #print(np.setxor1d(super_bad, utils._bad_idxs(transit_info["bad_idxs_user"])))
    #print()
    for ax, comp in zip(axes[i, :], comps):
        utils.plot_divided_wlcs(
            ax,
            data,
            t0=t0,
            ferr=0.001,
            comps_to_use=[comp],
            bad_idxs_user = transit_info["bad_idxs_user"],
            #bad_idxs_user = super_bad,
            #bad_idxs_user=np.setxor1d(super_bad, utils._bad_idxs(transit_info["bad_idxs_user"])),
            div_kwargs={"fmt": ".", "lw": 0.5, "mew": 0.0, "ms": 3, "c": colors[i]},
            bad_div_kwargs={"fmt": ".", "lw": 0.5, "mec": "k", "ms": 6, "c": "w"},
    )

fig.text(0.54, -0.02, "Time from estimated mid-transit (hours)", ha="center")
fig.text(-0.01, 0.5, "Normalizd flux", va="center", rotation="vertical")
fig.tight_layout()
fig.set_size_inches(FIG_LARGE)

utils.savefig("../paper/figures/extracted_wlcs/all_transits_extr_wlcs.pdf")

Transit 1
array([239, 240, 241, 256, 274, 275, 276, 277, 278, 279, 280, 281, 282,
       283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295,
       296, 297, 298])
[274 275 277 279 280 281 282 285 286 287 299]

Transit 2
array([ 1, 10])
[1]

Transit 3
array([ 1,  2,  3,  4,  5,  6,  7, 11])
[ 0  8  9 10 12 13 14 15 16 17]

Transit 4
array([ 93,  94, 149, 162, 163, 164, 165, 166, 167, 168, 169, 170])
[157 158 159 160 161 171]

Transit 5
array([  1,   2,   3,   4,   5,   6,  51,  52,  55,  56,  59,  64,  65,
        66,  67,  68,  72,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 108, 109, 110, 111, 112, 113, 114,
       120, 127, 128, 129, 130, 131, 132, 133, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180])
[  1   2   3  52  53  54  55  57  58  65  67  69  77  78  79  80  95  99
 100 115 116 117 118 119 127 128 132 166 167 168 169 170 171 172 173 181]



In [143]:
c1 = np.array([239, 240, 241, 256, 276, 277, 278, 283, 284, 285, 290, 296, 298])
u1 = utils._bad_idxs("[239, 240, 241, 256, 276, 278, 283, 284, 288:299]")
np.setxor1d(c1, u1)

array([277, 285, 288, 289, 291, 292, 293, 294, 295, 297, 299])

In [144]:
c2 = np.array([1, 10])
u2 = utils._bad_idxs("[10]")
np.setxor1d(c2, u2)

array([1])

In [145]:
c3 = np.array([1,  4,  5,  6, 11])
u3 = utils._bad_idxs("[0:17]")
np.setxor1d(c3, u3)

array([ 0,  2,  3,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17])